In [1]:
import qnmfits
import numpy as np
import scipy
import corner
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
from pathlib import Path

notebook_dir = Path().resolve()
sys.path.append(str(notebook_dir.parent))

notebook_dir = Path().resolve()
data_dir = notebook_dir.parent

from matplotlib.colors import to_hex
from matplotlib.lines import Line2D
from matplotlib.colors import LinearSegmentedColormap
from qnmfits.spatial_mapping_functions import * 
from bayes_qnm_GP_likelihood import *
from BGP_fits import BGP_fit
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [2]:
id = "0001"
sim_main = SXS_CCE(id, lev="Lev5", radius="R2")

n_max = 7

T0 = 17
T = 100

qnm_list = [(2,2,n,1) for n in np.arange(0, n_max+1)] #+ [(3,2,0,1)] + [(4,4,n,1) for n in np.arange(0, n_max+1)] + [(5,4,0,1)] 
spherical_modes = [(2, 2)] 

chif_mag_0 = sim_main.chif_mag
Mf_0 = sim_main.Mf

In [3]:
# Get the standard kernel parameters

with open(data_dir / 'bayes_qnm_GP_likelihood/data/tuned_params.pkl', 'rb') as f:
    params = pickle.load(f)

tuned_param_dict_main = params[id]

In [4]:
# The training / tuning is demonstrated in the `get_kernel_params` file and notebook. 

with open(data_dir / 'bayes_qnm_GP_likelihood/data/param_dict_sim_lm_full.pkl', 'rb') as f:
    param_dict_sim_lm = pickle.load(f)

tuning_hyperparams_s = [0.3239257812499994]  

hyperparam_rule_dict_s = {
    "sigma_max": "multiply",
}

tuned_param_dict_wn = {mode: get_new_params(param_dict_sim_lm[id][mode], tuning_hyperparams_s, hyperparam_rule_dict_s) for mode in param_dict_sim_lm[id]}

In [5]:
fit = BGP_fit(sim_main.times, 
            sim_main.h, 
            qnm_list, 
            Mf_0, 
            chif_mag_0, 
            T0, 
            tuned_param_dict_main, 
            kernel_main, 
            t0_method='closest', 
            T=T, 
            spherical_modes=spherical_modes,
            include_chif=True,
            include_Mf=True)

GP diagonal? False
_mask_data time: 0.0282 seconds
data_array creation time: 0.0047 seconds
model_term computation time: 0.0098 seconds
_get_exponential_terms time: 0.0530 seconds
_get_ls_amplitudes time: 0.0647 seconds
get_inverse_noise_covariance_matrix time: 0.1085 seconds
get_model_terms time: 0.2831 seconds
get_fisher_matrix time: 0.0431 seconds
get_b_vector time: 0.0540 seconds
mean_vector computation time: 0.0789 seconds
get_inverse (covariance_matrix) time: 0.0281 seconds
_get_samples time: 0.1607 seconds


[[ 3.89166781e-06  4.80253661e-08 -2.55434434e-05 -2.24157402e-05
   3.57387355e-05  1.68855352e-04  1.17508042e-04 -5.20486378e-04
  -4.08102628e-04  8.79428862e-04  4.64698478e-04 -8.35830614e-04
  -2.40288872e-04  4.11089243e-04  5.21286320e-05 -8.09461405e-05
   4.85605841e-06  3.73282630e-06]
 [ 4.80253661e-08  4.14606825e-06  2.20610464e-05 -2.88285774e-05
  -1.76843472e-04  6.28806612e-05  5.69525597e-04  9.32563136e-06
  -9.78397390e-04 -1.65452999e-04  9.11356542e-04  1.84958294e-04
  -4.30526242e-04 -8.70776103e-05  8.30470882e-05  2.00950263e-05
  -6.33483493e-07  4.50498263e-07]
 [-2.55434434e-05  2.20610464e-05  3.67977627e-04 -6.78643490e-06
  -1.76960683e-03 -1.14325026e-03  3.84570007e-03  5.51522056e-03
  -4.96029527e-03 -1.10463336e-02  4.05989211e-03  1.09498192e-02
  -1.81664607e-03 -5.38112445e-03  3.01505468e-04  1.09364617e-03
  -3.30209251e-05 -2.08679517e-05]
 [-2.24157402e-05 -2.88285774e-05 -6.78643490e-06  4.03123387e-04
   1.36187015e-03 -1.95383438e-03 -6.51560268e-03  4.39202201e-03
   1.31766070e-02 -6.13541718e-03 -1.31102437e-02  5.56262880e-03
   6.40811811e-03 -2.73914917e-03 -1.29488043e-03  5.02317101e-04
  -2.06018486e-05 -2.19277131e-05]
 [ 3.57387355e-05 -1.76843472e-04 -1.76960683e-03  1.36187015e-03
   1.41325090e-02 -9.43710962e-04 -4.58222053e-02 -1.43338987e-02
   7.98592877e-02  4.03062318e-02 -7.62582429e-02 -4.31751853e-02
   3.69529930e-02  2.16613593e-02 -7.16548914e-03 -4.69913307e-03
   6.59749275e-05  1.54162193e-05]
...
   1.54162193e-05  1.47500541e-04  1.52972456e-04 -4.30949731e-04
  -4.46366338e-04  7.10397774e-04  4.96213257e-04 -6.74216530e-04
  -2.55624209e-04  3.34505584e-04  5.45633515e-05 -6.59622038e-05
   5.03558871e-06  3.96945904e-06]]

In [6]:
print(fit.covariance_matrices[0])

[[ 3.89166781e-06  4.80253661e-08 -2.55434434e-05 -2.24157402e-05
   3.57387355e-05  1.68855352e-04  1.17508042e-04 -5.20486378e-04
  -4.08102628e-04  8.79428862e-04  4.64698478e-04 -8.35830614e-04
  -2.40288872e-04  4.11089243e-04  5.21286320e-05 -8.09461405e-05
   4.85605841e-06  3.73282630e-06]
 [ 4.80253661e-08  4.14606825e-06  2.20610464e-05 -2.88285774e-05
  -1.76843472e-04  6.28806612e-05  5.69525597e-04  9.32563136e-06
  -9.78397390e-04 -1.65452999e-04  9.11356542e-04  1.84958294e-04
  -4.30526242e-04 -8.70776103e-05  8.30470882e-05  2.00950263e-05
  -6.33483493e-07  4.50498263e-07]
 [-2.55434434e-05  2.20610464e-05  3.67977627e-04 -6.78643490e-06
  -1.76960683e-03 -1.14325026e-03  3.84570007e-03  5.51522056e-03
  -4.96029527e-03 -1.10463336e-02  4.05989211e-03  1.09498192e-02
  -1.81664607e-03 -5.38112445e-03  3.01505468e-04  1.09364617e-03
  -3.30209251e-05 -2.08679517e-05]
 [-2.24157402e-05 -2.88285774e-05 -6.78643490e-06  4.03123387e-04
   1.36187015e-03 -1.95383438e-03 -6.

In [7]:
T = T - T0 

fit_main = qnm_BGP_fit(
        sim_main.times,
        sim_main.h,
        qnm_list,
        Mf_0,
        chif_mag_0,
        T0,
        tuned_param_dict_main, 
        kernel_main, 
        t0_method="closest",
        T=T,
        spherical_modes=spherical_modes,
        include_chif=True,
        include_Mf=True,
    )

In [8]:
print(fit_main['covariance'])

[[ 3.89167014e-06  4.80184013e-08 -2.55434328e-05 -2.24156088e-05
   3.57390262e-05  1.68854254e-04  1.17505844e-04 -5.20482185e-04
  -4.08097464e-04  8.79419230e-04  4.64693976e-04 -8.35819303e-04
  -2.40287363e-04  4.11083099e-04  5.21283711e-05 -8.09448635e-05
   4.85599814e-06  3.73281187e-06]
 [ 4.80184013e-08  4.14606434e-06  2.20610462e-05 -2.88284925e-05
  -1.76843097e-04  6.28802195e-05  5.69523767e-04  9.32660161e-06
  -9.78393616e-04 -1.65454181e-04  9.11352621e-04  1.84959251e-04
  -4.30524216e-04 -8.70780808e-05  8.30466697e-05  2.00951101e-05
  -6.33483946e-07  4.50492963e-07]
 [-2.55434328e-05  2.20610462e-05  3.67976970e-04 -6.78698120e-06
  -1.76960449e-03 -1.14324317e-03  3.84570074e-03  5.51518929e-03
  -4.96030495e-03 -1.10462602e-02  4.05989829e-03  1.09497344e-02
  -1.81664451e-03 -5.38107896e-03  3.01504934e-04  1.09363668e-03
  -3.30204754e-05 -2.08678323e-05]
 [-2.24156088e-05 -2.88284925e-05 -6.78698120e-06  4.03121368e-04
   1.36186554e-03 -1.95382070e-03 -6.